## Advanced Lane Finding Project

The goals / steps of this project are the following:

1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
2. Apply a distortion correction to raw images.
3. Use color transforms, gradients, etc., to create a thresholded binary image.
4. Apply a perspective transform to rectify binary image ("birds-eye view").
5. Detect lane pixels and fit to find the lane boundary.
6. Determine the curvature of the lane and vehicle position with respect to center.
7. Warp the detected lane boundaries back onto the original image.
8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# Import statements
from collections import deque
from operator import sub
import numpy as np
import cv2
import glob
import os
import pickle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Imports needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
## 1. Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
## 2. Apply a distortion correction to raw images.

def get_distortion_matrices(img_size):
    # prepare object points for 9x6 chessboard
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list, search for chessboard corners, and populate objpoints & imgpoints
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

    # Save the camera calibration result for later use
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ))
    
    # return calibration matrix & distortion coeff
    return mtx, dist

# Function to undistort the image given the camera calibration parameters
def undistort(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

# Get calibration matrix and distortion coeff & Test undistortion on an image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])
mtx, dist = get_distortion_matrices(img_size)
dst = cv2.cvtColor(undistort(img, mtx, dist), cv2.COLOR_BGR2RGB)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)
plt.savefig('output_images/distortion_correction.jpg')

In [ ]:
# Applying distortion correction on example image
# Get calibration matrix and distortion coeff & Test undistortion on an image
img = cv2.imread('test_images/test3.jpg')
img_size = (img.shape[1], img.shape[0])
mtx, dist = get_distortion_matrices(img_size)
dst = cv2.cvtColor(undistort(img, mtx, dist), cv2.COLOR_BGR2RGB)

# Visualize undistortion
plt.figure(figsize = (60,45))
plt.imshow(dst)
plt.savefig('output_images/Undistorted_example.jpg')

In [ ]:
## 3. Use color transforms, gradients, etc., to create a thresholded binary image. Expects BGR image.

def color_grad_pipeline(img, s_thresh=(200, 255), sx_thresh=(30, 90)):
    img = np.copy(img)
    width_x = img.shape[1]
    height_y = img.shape[0]
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(sxbinary == 1) | (s_binary == 1)] = 1
    # Define Region threshold for a triangular region having base as the bottom of the image & height until the middle of the image
    left_bottom = (0, height_y)
    right_bottom = (width_x, height_y)
    apex = (width_x / 2.0, height_y / 1.75)

    fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
    fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
    # Region inside the triangle formed by the lines
    XX, YY = np.meshgrid(np.arange(width_x), np.arange(height_y))
    region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                        (YY > (XX*fit_right[0] + fit_right[1])) & \
                        (YY < (XX*fit_bottom[0] + fit_bottom[1]))
    # Region threshold masking
    combined_binary[~region_thresholds] = 0
    # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Applying Median Blur to smoothen out the pixels
    blurred_binary = cv2.medianBlur(combined_binary, 3)

    return color_binary, blurred_binary #combined_binary
    

image = cv2.imread('test_images/test5.jpg')
color_result, combined_result = color_grad_pipeline(image)

# Plot the result
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(40, 20))
f.tight_layout()

ax1.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image', fontsize=20)

ax2.imshow(color_result)
ax2.set_title('Pipeline Result', fontsize=20)

ax3.imshow(combined_result)
ax3.set_title('detected edges', fontsize=20)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/test6_binary.jpg')

In [ ]:
## 4. Apply a perspective transform to rectify binary image ("birds-eye view"). --- Functions to extract src points on the image

# Calculate histogram along x
def hist(img, start_pixel, end_pixel):
    (height, width) = img.shape
    start, end = height - end_pixel, height - start_pixel
    roi_img = img[start:end, :]

    histogram = np.sum(roi_img, axis=0)
    return histogram

# Get the source points for Perspective transform
def find_perspective_xform_pts(image):
    (height_y, width_x) = image.shape
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((image, image, image))
    # get the number of iterations to find the lane lines with sliding window of height 5 in the bottom half of the image
    n_iters = image.shape[0] // 10
    src_rect_pts = []
    # Taking the initial midpoint as the center of the image
    midpoint = np.int(width_x // 2)
    for iter in range(n_iters):
        # Take a histogram of the sliding window in the image
        histogram = hist(image, iter*5, (iter + 1)*5)
        # Check if the max value of the histogram exceeds a threshold, in this case, 5
        # print(np.amax(histogram[:midpoint]), np.amax(histogram[midpoint:]))
        found_points = (np.amax(histogram[:midpoint]) >= 5) & (np.amax(histogram[midpoint:]) >= 5)
        if found_points:
            # Find the peak of the left and right halves of the histogram
            # These will give points on the left and right lane lines
            left_x = np.argmax(histogram[:midpoint])
            right_x = np.argmax(histogram[midpoint:]) + midpoint
            # Append the source rectangle points
            src_rect_pts.append([left_x, height_y - iter*5])
            src_rect_pts.append([right_x, height_y - iter*5])
            # Move the mid point to the middle of the two identified pts
            midpoint = (left_x + right_x) // 2
    # Taking only the first and last pair. Need to shuffle the order of vertices
    src_pts = src_rect_pts[:2] + src_rect_pts[-1:] + src_rect_pts[-2:-1]
    return src_pts

# Process the image based on lane line colors and the region of interest. Expects an BGR image
def process_image(img):
    # color_result, combined_binary = pipeline(img)
    width_x = img.shape[1]
    height_y = img.shape[0]
    
    # Here, I'm doing relatively simplistic processing for the purpose of masking and identifying the lane lines. This is done just for the purpose of identifying source points on the image. 
    # Apply mask on expected lane line colors- white & yellow
    # White mask 
    lower_white = np.array([200,200,200])
    upper_white = np.array([255,255,255])
    white_mask = cv2.inRange(img, lower_white, upper_white)
    # Yellow mask
    lower_yellow = np.array([0,80,130])
    upper_yellow = np.array([85,255,255])
    yellow_mask = cv2.inRange(img, lower_yellow, upper_yellow)

    lane_mask = white_mask + yellow_mask

    img[lane_mask == 0] = [0, 0, 0]

    # Define Region threshold for a triangular region having base as the bottom of the image & height until the middle of the image
    left_bottom = (0, height_y)
    right_bottom = (width_x, height_y)
    apex = (width_x / 2.0, height_y / 1.7)

    fit_left = np.polyfit((left_bottom[0], apex[0]), (left_bottom[1], apex[1]), 1)
    fit_right = np.polyfit((right_bottom[0], apex[0]), (right_bottom[1], apex[1]), 1)
    fit_bottom = np.polyfit((left_bottom[0], right_bottom[0]), (left_bottom[1], right_bottom[1]), 1)
    # Region inside the triangle formed by the lines
    XX, YY = np.meshgrid(np.arange(width_x), np.arange(height_y))
    region_thresholds = (YY > (XX*fit_left[0] + fit_left[1])) & \
                        (YY > (XX*fit_right[0] + fit_right[1])) & \
                        (YY < (XX*fit_bottom[0] + fit_bottom[1]))
    # Region threshold masking
    img[~region_thresholds] = 0
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Applying Median Blur to smoothen out the pixels
    blurred_gray = cv2.medianBlur(gray, 3)
    return blurred_gray

In [ ]:
# Load test image
img = cv2.imread('test_images/test1.jpg')
# Undistort the image
undistorted_img = undistort(img, mtx, dist)

combined_binary = process_image(undistorted_img)
# plt.imshow(combined_binary)
src_pts = find_perspective_xform_pts(combined_binary)
print(src_pts)
src_img = cv2.polylines(img, np.array([src_pts], dtype=int), True, (0, 0, 255), 5)
plt.imshow(cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB))

In [ ]:
## 4. Apply a perspective transform to rectify binary image ("birds-eye view") ---- Find the perspective transform matrix and persist

def get_perspective_matrices(img, src_pts, dst_pts):
    # 1) use cv2.getPerspectiveTransform() to get M, the transform matrix
    # 2) use cv2.warpPerspective() to warp the image to a top-down view
    img_size = (img.shape[1], img.shape[0])
    
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    Minv = cv2.getPerspectiveTransform(dst_pts, src_pts)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)
    return warped, M, Minv

def xform_perspective(img, xform_matrix):
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, xform_matrix, img_size)
    
# Read in the saved camera matrix and distortion coefficients
dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Read in test image
# One set of perspective transform done on the image straight_lines1.jpg
# img = cv2.imread('test_images/straight_lines1.jpg')
# src_pts = np.float32([[260, 680], [1048, 680], [833, 550], [460, 550]])
# dst_pts = np.float32([[360, 680], [948, 680], [933, 350], [360, 350]])

img = cv2.imread('test_images/test1.jpg')
img_size = (img.shape[1], img.shape[0])

# Undistort the image
undistorted_img = undistort(img, mtx, dist)

# Source points identified in previous step. The destination points are approximated based on judgement
src_pts = np.float32([[281, 695], [1104, 695], [836, 530], [495, 530]])
dst_pts = np.float32([[400, 700], [978, 700], [978, 300], [400, 300]])
src_img = cv2.polylines(undistorted_img, np.array([src_pts], dtype=int), True, (0, 0, 255), 5)

top_down, perspective_M, perspective_Minv = get_perspective_matrices(src_img, src_pts, dst_pts)

# Save the perspective xform metrices for later use
perspective_pickle = {}
perspective_pickle["mtxM"] = perspective_M
perspective_pickle["mtxMinv"] = perspective_Minv
pickle.dump(perspective_pickle, open("camera_cal/perspective_pickle.p", "wb" ))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image with Src Pts', fontsize=30)
ax2.imshow(cv2.cvtColor(top_down, cv2.COLOR_BGR2RGB))
ax2.set_title('Warped Image with Dest Pts', fontsize=30)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.savefig('output_images/perspective_transform.jpg')

In [ ]:
test1 = cv2.imread('test_images/test1.jpg')
undist_img = undistort(test1, mtx, dist)
_, combined_binary = color_grad_pipeline(undist_img)
# Read in the saved camera matrix and distortion coefficients
perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb" ) )
perspective_M = perspective_pickle["mtxM"]
binary_warped = xform_perspective(combined_binary, perspective_M)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
ax1.imshow(binary_warped)
ax2.imshow(cv2.cvtColor(undist_img, cv2.COLOR_BGR2RGB))
ax3.imshow(combined_binary)

In [ ]:
## 5. Detect lane pixels and fit to find the lane boundary.

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom three quarters of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//4:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 50
    # Set minimum number of pixels found to recenter window
    minpix = 30

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # # Draw the windows on the visualization image
        # cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        # (win_xleft_high,win_y_high),(0,255,0), 2) 
        # cv2.rectangle(out_img,(win_xright_low,win_y_low),
        # (win_xright_high,win_y_high),(0,255,0), 2) 

        ### Identify the nonzero pixels in x and y within the window ###
        win_y_mask = np.array((nonzeroy >= win_y_low) & (nonzeroy < win_y_high))
        win_xleft_mask = np.array((nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high))
        win_xright_mask = np.array((nonzerox >= win_xright_low) & (nonzerox < win_xright_high))
        good_left_inds = (nonzeroy[win_y_mask & win_xleft_mask], 
                          nonzerox[win_y_mask & win_xleft_mask])
        good_right_inds = (nonzeroy[win_y_mask & win_xright_mask], 
                          nonzerox[win_y_mask & win_xright_mask])
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### If found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(np.array(good_left_inds[1])) > minpix:
            leftx_current = np.rint(np.mean(good_left_inds[1])).astype(int)
        if len(np.array(good_right_inds[1])) > minpix:
            rightx_current = np.rint(np.mean(good_right_inds[1])).astype(int)

    # Extract left and right line pixel positions
    leftx = np.concatenate([it[1] for it in left_lane_inds])
    lefty = np.concatenate([it[0] for it in left_lane_inds]) 
    rightx = np.concatenate([it[1] for it in right_lane_inds])
    righty = np.concatenate([it[0] for it in right_lane_inds])
    return leftx, lefty, rightx, righty

def shade_lane(binary_warped, left_fitx_end, left_fitx_start, right_fitx_end, right_fitx_start, y_end, y_start, xm_per_pix):
    lane_polylines = np.zeros_like(np.dstack((binary_warped, binary_warped, binary_warped)))
    cv2.fillPoly(lane_polylines, 
                 np.array([[
                    (left_fitx_end / xm_per_pix, y_end), 
                    (left_fitx_start / xm_per_pix, y_start), 
                    (right_fitx_start / xm_per_pix, y_start), 
                    (right_fitx_end / xm_per_pix, y_end)
                    ]], dtype=np.int32), 
                 color=[0, 255, 0])
    return lane_polylines

# Function to plot the identified lane lines on the binary image
def plot_polynomials(binary_warped, leftx, lefty, rightx, righty, left_fitx, right_fitx):
    out_img = np.zeros_like(np.dstack((binary_warped, binary_warped, binary_warped)))
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    cv2.polylines(out_img, np.array([np.float32(np.stack([left_fitx, ploty], axis=1))], dtype=int), False, color=[0, 255, 0], thickness=2)
    cv2.polylines(out_img, np.array([np.float32(np.stack([right_fitx, ploty], axis=1))], dtype=int), False, color=[0, 255, 0], thickness=2)

    return out_img

# Fit polynomial on the lane pixels identified from the sliding window approach
def fit_polynomial(binary_warped, ym_per_pix = 1, xm_per_pix = 1):
    # Find the lane pixels first
    leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)
    ### Fit a second order polynomial to x,y in world space #####
    left_fit = np.polyfit(ym_per_pix * np.asarray(lefty, dtype=np.int), xm_per_pix * np.asarray(leftx, dtype=np.int), 2)
    right_fit = np.polyfit(ym_per_pix * np.asarray(righty, dtype=np.int), xm_per_pix * np.asarray(rightx, dtype=np.int), 2)
    #print(left_fit, right_fit)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*(ploty*ym_per_pix)**2 + left_fit[1]*(ploty*ym_per_pix) + left_fit[2]
        right_fitx = right_fit[0]*(ploty*ym_per_pix)**2 + right_fit[1]*(ploty*ym_per_pix) + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*(ploty*ym_per_pix)**2 + 1*(ploty*ym_per_pix)
        right_fitx = 1*(ploty*ym_per_pix)**2 + 1*(ploty*ym_per_pix)
    
    # Find mid point of the detected lane lines in world space for offset calculation
    midpoint_meters = (left_fitx[-1] + right_fitx[-1]) / 2
    #print(left_fitx[-1], right_fitx[-1], midpoint_meters)

    return left_fitx, right_fitx, ploty, left_fit, right_fit, midpoint_meters



test_img = cv2.imread('test_images/test4.jpg')
undist_img = undistort(test_img, mtx, dist)
_, combined_binary = color_grad_pipeline(undist_img)
# Read in the saved camera matrix and distortion coefficients
perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb" ) )
perspective_M = perspective_pickle["mtxM"]
binary_warped = xform_perspective(combined_binary, perspective_M)
# perspective_Minv = perspective_pickle["mtxMinv"]
left_fitx, right_fitx, ploty, _, _, _ = fit_polynomial(binary_warped)
out_img = shade_lane(binary_warped, left_fitx[0], left_fitx[-1], 
                     right_fitx[0], right_fitx[-1], ploty[0], ploty[-1], xm_per_pix=1)
# unwarped_polylines = xform_perspective(out_img, perspective_Minv)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
ax1.imshow(out_img)
ax2.imshow(combined_binary) #unwarped_polylines
ax3.imshow(binary_warped)

In [ ]:
test_img = cv2.imread('test_images/test5.jpg')
undist_img = undistort(test_img, mtx, dist)
_, combined_binary = color_grad_pipeline(undist_img)
# Read in the saved camera matrix and distortion coefficients
perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb" ) )
perspective_M = perspective_pickle["mtxM"]
binary_warped = xform_perspective(combined_binary, perspective_M)
# perspective_Minv = perspective_pickle["mtxMinv"]
leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)
left_fitx, right_fitx, _, _, _, _ = fit_polynomial(binary_warped)
out_img = plot_polynomials(binary_warped, leftx, lefty, rightx, righty, left_fitx, right_fitx)
plt.figure(figsize = (60,45))
plt.imshow(out_img)
plt.savefig("output_images/test5_lane_polylines.jpg")

In [ ]:
## Look ahead Filter implementation - look for lane pixel around the previous poly
def fit_poly(img_shape, leftx, lefty, rightx, righty, ym_per_pix, xm_per_pix):
    ### Fit a second order polynomial to x,y in world space #####
    left_fit = np.polyfit(ym_per_pix * np.asarray(lefty, dtype=np.int), xm_per_pix * np.asarray(leftx, dtype=np.int), 2)
    right_fit = np.polyfit(ym_per_pix * np.asarray(righty, dtype=np.int), xm_per_pix * np.asarray(rightx, dtype=np.int), 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    ### Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*(ploty*ym_per_pix)**2 + left_fit[1]*(ploty*ym_per_pix) + left_fit[2]
    right_fitx = right_fit[0]*(ploty*ym_per_pix)**2 + right_fit[1]*(ploty*ym_per_pix) + right_fit[2]
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty


def search_around_poly(binary_warped, ym_per_pix, xm_per_pix, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values within the +/- margin of our polynomial function ###
    ### This is in pixel space. NEed the multipliers coz the polynomial coefficients are generated on points in world space
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = (left_fit[0]*(ploty*ym_per_pix)**2 + left_fit[1]*(ploty*ym_per_pix) + left_fit[2]) / xm_per_pix
    right_fitx = (right_fit[0]*(ploty*ym_per_pix)**2 + right_fit[1]*(ploty*ym_per_pix) + right_fit[2]) / xm_per_pix

    # print(left_fit, right_fit)
    left_lane_inds = []
    right_lane_inds = []
    for i in range(len(ploty)):
        y_mask = np.array(nonzeroy == ploty[i])
        xleft_mask = np.array((nonzerox >= left_fitx[i] - margin) & 
                              (nonzerox < left_fitx[i] + margin))
        xright_mask = np.array((nonzerox >= right_fitx[i] - margin) & 
                                (nonzerox < right_fitx[i] + margin))                     
        left_lane_inds.append((nonzeroy[y_mask & xleft_mask], 
                               nonzerox[y_mask & xleft_mask]))
        right_lane_inds.append((nonzeroy[y_mask & xright_mask], 
                                nonzerox[y_mask & xright_mask]))
    
    # Again, extract left and right line pixel positions
    leftx = np.concatenate([it[1] for it in left_lane_inds])
    lefty = np.concatenate([it[0] for it in left_lane_inds]) 
    rightx = np.concatenate([it[1] for it in right_lane_inds])
    righty = np.concatenate([it[0] for it in right_lane_inds])
    # print(len(leftx), len(lefty), len(rightx), len(righty))
    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty, ym_per_pix, xm_per_pix)
    
    # Find mid point of the detected lane lines in world space for offset calculation
    midpoint_meters = (left_fitx[-1] + right_fitx[-1]) / 2
    #print(left_fitx[-1], right_fitx[-1], midpoint_meters)

    return left_fitx, right_fitx, ploty, left_fit, right_fit, midpoint_meters

In [ ]:
## 6. Determine the curvature of the lane and vehicle position with respect to center.
## 7. Warp the detected lane boundaries back onto the original image.
## 8. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
def weighted_img(img, initial_img, α=0.8, β=1, γ=0.):
    """
    initial_img * α + img * β + γ
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

# # This function is NOT Being used!!!
# def overlay_transparent(background, overlay, x=0, y=0):

#     background_width = background.shape[1]
#     background_height = background.shape[0]

#     if x >= background_width or y >= background_height:
#         return background

#     h, w = overlay.shape[0], overlay.shape[1]

#     if x + w > background_width:
#         w = background_width - x
#         overlay = overlay[:, :w]

#     if y + h > background_height:
#         h = background_height - y
#         overlay = overlay[:h]

#     if overlay.shape[2] < 4:
#         overlay = np.concatenate(
#             [
#                 overlay,
#                 np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
#             ],
#             axis = 2,
#         )

#     overlay_image = overlay[..., :3]
#     mask = overlay[..., 3:] / (255.0 * 7)

#     background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

#     return background

def measure_curvature_real(binary_warped):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Get the polynomial lines fitted on the detected lane lines
    left_fitx, right_fitx, ploty, left_fit_cr, right_fit_cr, midpoint_meters = fit_polynomial(binary_warped, ym_per_pix, xm_per_pix)
    lane_polylines = shade_lane(binary_warped, left_fitx[0], left_fitx[-1], 
                                right_fitx[0], right_fitx[-1], ploty[0], ploty[-1], xm_per_pix)

    # Define y-value for radius of curvature measurement
    # Choose the max y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2 * left_fit_cr[0] * ym_per_pix*y_eval + left_fit_cr[1])**2)**1.5) / (2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * ym_per_pix*y_eval + right_fit_cr[1])**2)**1.5) / (2 * right_fit_cr[0])
    
    # Find offset = (midpoint of Image in meters - midpoint of lane in meters)  
    offset_meters = (binary_warped.shape[1] * xm_per_pix / 2) - midpoint_meters
    # print(binary_warped.shape[1], binary_warped.shape[1] * xm_per_pix / 2, midpoint_meters)
    return left_fitx, right_fitx, left_curverad, right_curverad, offset_meters, lane_polylines

In [ ]:
## End-to-end pipeline identifying the lane and overlaying it on the input image
def run_image_pipeline(img_fname, mtx, dist, perspective_M, perspective_Minv):

    input_img = cv2.imread(img_fname)

    # Undistort the image
    undistorted_img = undistort(input_img, mtx, dist)

    # Apply color & gradient thresholds to get the binary image
    _, combined_binary = color_grad_pipeline(undistorted_img)

    # Apply Perspective xform
    binary_warped = xform_perspective(combined_binary, perspective_M)

    # Identify lane polylines & get the radius of curvature and offset in meters for both lane lines
    left_fitx, right_fitx, left_curverad, right_curverad, offset_meters, lane_polylines = measure_curvature_real(binary_warped)

    # Unwarp the image with lane polylines
    unwarped_polylines = xform_perspective(lane_polylines, perspective_Minv)

    # Combine the result with the original image
    overlaid_img = cv2.addWeighted(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB), 1, unwarped_polylines, 0.3, 0)
    # overlaid_img = overlay_transparent(cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB), unwarped_polylines)

    # Add text to the image - Take minimum of the left & right radii
    geolocation_text = "Radius of Curvature: {}m".format(round(min([abs(left_curverad), abs(right_curverad)])))
    offset_text = "Vehicle is {} meters {} of center".format(abs(round(offset_meters, 2)), 
                "left" if offset_meters > 0 else "right")
    # Add text 
    overlaid_img = cv2.putText(overlaid_img, geolocation_text, (400, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                               1, 255, 2, cv2.LINE_AA) 
    overlaid_img = cv2.putText(overlaid_img, offset_text, (360, 100), cv2.FONT_HERSHEY_SIMPLEX,  
                               1, 255, 2, cv2.LINE_AA) 
    
    # Display and save
    plt.figure(figsize = (60,45))
    plt.imshow(overlaid_img)
    # Save the output
    plt.savefig('output_images/{}_final.jpg'.format(fname.split('/')[-1].split('.')[0]))

# Run entire pipeline on the test images
images = glob.glob('test_images/*.jpg')
# Read in the saved camera matrix, distortion coefficients, and perspective transform matrices
dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb"))
mtx =  dist_pickle["mtx"]
dist = dist_pickle["dist"]
perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb"))
perspective_M = perspective_pickle["mtxM"]
perspective_Minv = perspective_pickle["mtxMinv"]
# Iterate through the test images
for idx, fname in enumerate(images):
    run_image_pipeline(fname, mtx, dist, perspective_M, perspective_Minv)

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values at the bottom of the image for the last 5 fits of the line
        self.recent_xfitted = deque(maxlen=5) 
        # Average x values at the bottom of the image for the fitted line over the last 5 iterations
        self.bestx = None     
        # x values at the top of the image for the last 5 fits of the line
        self.recent_xfitted_top = deque(maxlen=5) 
        # Average x values at the top of the image for the fitted line over the last 5 iterations
        self.bestx_top = None     
        # polynomial coefficients of the last 5 fits of the line
        self.recent_fit = deque(maxlen=5) 
        #polynomial coefficients averaged over the last 5 iterations
        self.best_fit = None  
        # polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        # radius of curvature of the line in meters
        self.radius_of_curvature = None 
        # distance in meters of vehicle center from the line
        self.line_base_pos = None 
        # difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        # x values for detected line pixels
        self.allx = None  
        # y values for detected line pixels
        self.ally = None
        # count of consecutive failures in lane detection as per the sanity checks
        self.failures = 0 
        # Frame number - which frame this belongs to
        self.current_frame = 0
    
## Sanity check methods - 
#  1. Checking that they have similar curvature
def is_curvrad_similar(left_rad, right_rad, rad_thresh = 1000):
    '''Compares the radius of curvature of left & right lane lines for similarity against a baseline threshold'''
    return abs(left_rad - right_rad) <= rad_thresh

#  2. Checking that they are separated by approximately the right distance horizontally 
def is_right_width(left_x, right_x, width_baseline = 3.0, diff_threshold = 0.2):
    '''Compares the distance between the two lines at the bottom of the image against a baseline width = 3.7 meters & error threshold of 0.2 meters'''
    return abs(abs(left_x - right_x) - width_baseline) <= diff_threshold

#  3. Checking that they are roughly parallel
def is_parallel(left_cr, right_cr, sec_order_thresh = 0.0006, first_order_thresh = 0.003):
    '''Compares the two lines for being parallel against a baseline difference for second order coeff = 0.0003 & first order coeff = 0.3'''
    return (abs(left_cr[0] - right_cr[0]) <= sec_order_thresh) & \
           (abs(left_cr[1] - right_cr[1]) <= first_order_thresh)


# Function to update Line class variables once lane line identification pipeline runs
def update_line(line, fitx, fit_cr, ploty, curverad, vehicle_dist, prev_exists, failure_count, current_frame):
    # was the line detected in the last iteration?
    line.detected = prev_exists
    # x values at bottom of the image for the last 5 fits of the line
    line.recent_xfitted.append(fitx[-1]) 
    # average x values at bottom of the image for the fitted line over the last 5 iterations
    line.bestx = np.mean(line.recent_xfitted, dtype=type(line.recent_xfitted[0]))     
    # x values at top of the image for the last 5 fits of the line
    line.recent_xfitted_top.append(fitx[0]) 
    #average x values of the fitted line over the last 5 iterations
    line.bestx_top = np.mean(line.recent_xfitted_top, dtype=type(line.recent_xfitted_top[0]))     
    # polynomial coefficients of the last 5 fits of the line
    line.recent_fit.append(fit_cr) 
    # polynomial coefficients averaged over the last 5 iterations
    line.best_fit = np.mean(line.recent_fit, axis=1)
    # polynomial coefficients for the most recent fit
    line.current_fit = fit_cr  
    # radius of curvature of the line in meters
    line.radius_of_curvature = curverad
    # distance in meters of vehicle center from the line
    line.line_base_pos = vehicle_dist 
    # difference in fit coefficients between last and new fits
    line.diffs = np.array([0,0,0], dtype='float') if len(line.recent_fit) < 2 else  list(map(sub, line.recent_fit[-1], line.recent_fit[-2]))
    # x values for detected line pixels
    line.allx = fitx 
    # y values for detected line pixels
    line.ally = ploty
    # count of consecutive failures in lane detection as per the sanity checks
    line.failures = failure_count 
    # Frame number - which frame this belongs to
    line.current_frame = current_frame

# Function to increment the variable failures when the sanity check fails
def increment_failure(line, failure_count, current_frame):
    line.failures = failure_count
    line.current_frame = current_frame


In [ ]:
## End-to-end pipeline for video feed identifying the lane and overlaying it on each frame
def get_frame_lanes(binary_warped, isFirstFrame, left_fit=None, right_fit=None):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Get the polynomial lines fitted on the detected lane lines
    if isFirstFrame:
        # Call the function that implements sliding window approach
        left_fitx, right_fitx, ploty, left_fit_cr, right_fit_cr, midpoint_meters = fit_polynomial(binary_warped, ym_per_pix, xm_per_pix)
    else:
        # Call the function with Look ahead implementation
        left_fitx, right_fitx, ploty, left_fit_cr, right_fit_cr, midpoint_meters = search_around_poly(binary_warped, ym_per_pix, xm_per_pix, left_fit, right_fit)    
    # Define y-value for radius of curvature measurement
    # Choose the max y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2 * left_fit_cr[0] * ym_per_pix*y_eval + left_fit_cr[1])**2)**1.5) / (2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * ym_per_pix*y_eval + right_fit_cr[1])**2)**1.5) / (2 * right_fit_cr[0])
    
    # Find offset = (midpoint of Image in meters - midpoint of lane in meters)  
    offset_meters = (binary_warped.shape[1] * xm_per_pix / 2) - midpoint_meters
    # print(binary_warped.shape[1], binary_warped.shape[1] * xm_per_pix / 2, midpoint_meters)
    return left_fitx, right_fitx, left_fit_cr, right_fit_cr, left_curverad, right_curverad, offset_meters



In [ ]:
def run_frame_pipeline(input_frame, mtx, dist, perspective_M, perspective_Minv, left_lane, right_lane):
    # Convert the RGB image to BGR
    input_img = cv2.cvtColor(input_frame, cv2.COLOR_RGB2BGR)
    # Undistort the image
    undistorted_img = undistort(input_img, mtx, dist)

    # Apply color & gradient thresholds to get the binary image
    _, combined_binary = color_grad_pipeline(undistorted_img)

    # Apply Perspective xform
    binary_warped = xform_perspective(combined_binary, perspective_M)

    # Defining the coeff of transformation for x dimension from pixel space to world space
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # define y points
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])

    # get the cumulative value of failure_count until last frame from any of the lane objects
    failure_count = left_lane.failures
    # print(failure_count)

    # get the previous frame number from any of the lane objects
    current_frame = left_lane.current_frame 
    isFirstFrame = (current_frame == 0)

    # Identify lane polylines & get the radius of curvature and offset in meters for both lane lines
    if isFirstFrame or failure_count > 3: # either the first frame or the lane detection has failed 3 times consecutively
        # get_frame_lanes Function takes only the image. Doesn't need the polynomial coeffs. from the prev fit.
        left_fitx, right_fitx, left_fit_cr, right_fit_cr, left_curverad, right_curverad, \
        offset_meters = get_frame_lanes(binary_warped, True) 
        # print(left_fitx[-1], right_fitx[-1], left_fit_cr, right_fit_cr)
        # Calculate distance of vehicle center from the lane line
        lane_center = (left_fitx[-1] + right_fitx[-1]) / 2
        vehicle_dist_left = lane_center + offset_meters - left_fitx[-1]
        vehicle_dist_right = right_fitx[-1] - lane_center - offset_meters
        # Sanity check on the detected lane lines
        sanity_checked = is_right_width(left_fitx[-1], right_fitx[-1], width_baseline = 3.0, diff_threshold = 0.4) & \
                         is_parallel(left_fit_cr, right_fit_cr, sec_order_thresh = 0.0009, first_order_thresh = 0.03)
                         #is_curvrad_similar(left_curverad, right_curverad, rad_thresh = 5000) & \
                         
        # print(sanity_checked)
        if isFirstFrame or sanity_checked:
            # lines are good. Update the variables of the Line objects. Failures variable to be reset to zero.
            update_line(left_lane, left_fitx, left_fit_cr, ploty, left_curverad, vehicle_dist_left, False, 0, current_frame + 1)
            update_line(right_lane, right_fitx, right_fit_cr, ploty, right_curverad, vehicle_dist_right, False, 0, current_frame + 1)
            # lines are good. Use the average of last 5 iterations' x location & polynomial coeffs.
            # This is for smoothening out the identified lanes
            lane_polylines = shade_lane(binary_warped, left_lane.bestx_top, left_lane.bestx, right_lane.bestx_top, \
                                        right_lane.bestx, ploty[0], ploty[-1], xm_per_pix)
        else:
            # Need to use previous lines' parameters & increment failure_count
            # Use the average of previous 5 iterations' x location & polynomial coeffs. This is for smoothening out the identified lanes
            lane_polylines = shade_lane(binary_warped, left_lane.bestx_top, left_lane.bestx, right_lane.bestx_top, \
                                        right_lane.bestx, ploty[0], ploty[-1], xm_per_pix)
            # Increment failure count & frame number for both lane objects
            increment_failure(left_lane, failure_count + 1, current_frame + 1)
            increment_failure(right_lane, failure_count + 1, current_frame + 1)

    else:
        # get_frame_lanes Function takes the image + the polynomial coeffs. from the prev fit.
        left_fitx, right_fitx, left_fit_cr, right_fit_cr, left_curverad, right_curverad, \
        offset_meters = get_frame_lanes(binary_warped, isFirstFrame, left_lane.current_fit, right_lane.current_fit)
        # Calculate distance of vehicle center from the lane line
        lane_center = (left_fitx[-1] + right_fitx[-1]) / 2
        vehicle_dist_left = lane_center + offset_meters - left_fitx[-1]
        vehicle_dist_right = right_fitx[-1] - lane_center - offset_meters
        # print(left_fitx[-1], right_fitx[-1], left_fit_cr, right_fit_cr)
        # Sanity check on the detected lane lines
        sanity_checked = is_right_width(left_fitx[-1], right_fitx[-1], width_baseline = 3.0, diff_threshold = 0.4) & \
                         is_parallel(left_fit_cr, right_fit_cr, sec_order_thresh = 0.0009, first_order_thresh = 0.03)
                         #is_curvrad_similar(left_curverad, right_curverad, rad_thresh = 5000) & \
                         
        # print(sanity_checked)
        if sanity_checked:
            # lines are good. update the variables of the line objects. Failures variable to be reset to zero.
            update_line(left_lane, left_fitx, left_fit_cr, ploty, left_curverad, vehicle_dist_left, \
                       (left_lane.failures == 0), 0, current_frame + 1)
            update_line(right_lane, right_fitx, right_fit_cr, ploty, right_curverad, vehicle_dist_right, \
                       (right_lane.failures == 0), 0, current_frame + 1)
            # lines are good. Use the average of last 5 iterations' x location & polynomial coeffs.
            # This is for smoothening out the identified lanes
            lane_polylines = shade_lane(binary_warped, left_lane.bestx_top, left_lane.bestx, right_lane.bestx_top, \
                                        right_lane.bestx, ploty[0], ploty[-1], xm_per_pix)
        else:
            # Need to use previous lines' parameters & increment failure_count
            # Use the average of previous 5 iterations' x location & polynomial coeffs. This is for smoothening out the identified lanes
            lane_polylines = shade_lane(binary_warped, left_lane.bestx_top, left_lane.bestx, right_lane.bestx_top, \
                                        right_lane.bestx, ploty[0], ploty[-1], xm_per_pix)
            # Increment failure count & frame number for both lane objects
            increment_failure(left_lane, failure_count + 1, current_frame + 1)
            increment_failure(right_lane, failure_count + 1, current_frame + 1)

    # Unwarp the image with lane polylines
    unwarped_polylines = xform_perspective(lane_polylines, perspective_Minv)

    # Combine the result with the original image
    overlaid_img = cv2.addWeighted(input_frame, 1, unwarped_polylines, 0.3, 0)
    # overlay_transparent(input_frame, unwarped_polylines)

    # Add text to the image - Radii of curvature & offset
    geolocation_text = "Radius of Curvature - Left: {}m, Right: {}m".format(round(abs(left_curverad)), round(abs(right_curverad)))
    offset_text = "Vehicle is {} meters {} of center".format(abs(round(offset_meters, 2)), 
                "left" if offset_meters > 0 else "right")
    # Add text 
    overlaid_img = cv2.putText(overlaid_img, geolocation_text, (260, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                               1, 255, 2, cv2.LINE_AA) 
    overlaid_img = cv2.putText(overlaid_img, offset_text, (360, 100), cv2.FONT_HERSHEY_SIMPLEX,  
                               1, 255, 2, cv2.LINE_AA) 

    return overlaid_img

# Wrapper function for VideoFileClip.fl_image
def process_frame(frame):
    return run_frame_pipeline(frame, mtx, dist, perspective_M, perspective_Minv, left_lane, right_lane)


In [ ]:
# Running the entire pipeline on video input
try:
    dist_pickle, perspective_pickle    
except NameError:
    # Read in the saved camera matrix, distortion coefficients, and perspective transform matrices
    dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb"))
    mtx =  dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb"))
    perspective_M = perspective_pickle["mtxM"]
    perspective_Minv = perspective_pickle["mtxMinv"]
# Instantiate the left and right lane classes
left_lane = Line()
right_lane = Line()

project_output = 'test_videos_output/project_video.mp4'
vid_clip = VideoFileClip("project_video.mp4") #.subclip(37,41)
# Run the lane finding algo on every frame in the clip
processed_clip = vid_clip.fl_image(process_frame)
%time processed_clip.write_videofile(project_output, audio=False)

In [ ]:
# Running the entire pipeline on challenge video input
try:
    dist_pickle, perspective_pickle    
except NameError:
    # Read in the saved camera matrix, distortion coefficients, and perspective transform matrices
    dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb"))
    mtx =  dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb"))
    perspective_M = perspective_pickle["mtxM"]
    perspective_Minv = perspective_pickle["mtxMinv"]
# Instantiate the left and right lane classes
left_lane = Line()
right_lane = Line()

challenge_output = 'test_videos_output/challenge_video.mp4'
vid_clip = VideoFileClip("challenge_video.mp4") #.subclip(0,5)
# Run the lane finding algo on every frame in the clip
processed_clip = vid_clip.fl_image(process_frame)
%time processed_clip.write_videofile(challenge_output, audio=False)

In [ ]:
# Running the entire pipeline on harder challenge video input
try:
    dist_pickle, perspective_pickle    
except NameError:
    # Read in the saved camera matrix, distortion coefficients, and perspective transform matrices
    dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb"))
    mtx =  dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    perspective_pickle = pickle.load( open( "camera_cal/perspective_pickle.p", "rb"))
    perspective_M = perspective_pickle["mtxM"]
    perspective_Minv = perspective_pickle["mtxMinv"]
# Instantiate the left and right lane classes
left_lane = Line()
right_lane = Line()

challenge_output_2 = 'test_videos_output/harder_challenge_video.mp4'
vid_clip = VideoFileClip("harder_challenge_video.mp4") #.subclip(0,5)
# Run the lane finding algo on every frame in the clip
processed_clip = vid_clip.fl_image(process_frame)
%time processed_clip.write_videofile(challenge_output_2, audio=False)